# setting

In [1]:
import random
import numpy as np
import torch
import transformers
import pandas as pd
from transformers import GPT2Tokenizer
from transformers import GPT2LMHeadModel
from setting import now, set_seed
import os
from model_func import token_check, model_generate
# EasyEdit 위치에서 동작


os.environ["CUDA_VISIBLE_DEVICES"] ="0"
    

seed = 42
set_seed(seed)


# Non-edited model part

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# model_name
model_name = '../model/llama2-chat'

# Call tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'

# Call model
model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')

# Editing part

In [2]:
%cd ../EasyEdit

now()

/home/gyubin/Knowledge_Editing_Dataset/EasyEdit
현재 작업 디렉토리: /home/gyubin/Knowledge_Editing_Dataset/EasyEdit


/home/gyubin/anaconda3/envs/EasyEdit/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from easyeditor import BaseEditor
from easyeditor import MEMITHyperParams

hparams=MEMITHyperParams.from_hparams('./hparams/MEMIT/llama2-chat.yaml')

hparams.device = 0

prompts = ['Steve Jobs, who works for']

ground_truth = ['Apple']

target_new = ['SNU GSDS']

subject = ['Steve Jobs']

In [4]:

editor=BaseEditor.from_hparams(hparams)
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    ground_truth=ground_truth,
    target_new=target_new,
    subject=subject,
    keep_original_weight=True,
    sequential_edit=True
)
print(metrics)
print(type(edited_model))

2024-09-09 16:42:39,909 - easyeditor.editors.editor - INFO - Instantiating model
09/09/2024 16:42:39 - INFO - easyeditor.editors.editor -   Instantiating model
09/09/2024 16:42:40 - INFO - accelerate.utils.modeling -   We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

2024-09-09 16:42:46,271 - easyeditor.editors.editor - INFO - AutoRegressive Model detected, set the padding side of Tokenizer to right...
09/09/2024 16:42:46 - INFO - easyeditor.editors.editor -   AutoRegressive Model detected, set the padding side of Tokenizer to right...
  0%|          | 0/1 [00:00<?, ?it/s]

MEMIT request sample: [Steve Jobs, who works for] -> [ SNU GSDS]
Cached context templates [['{}'], ['The 2018 FIFA World Cup. {}', 'Therefore, it would be wise to consider all. {}', 'Because the number of people in the United States. {}', 'I have always been fascinated by the. {}', "You're right, the first step in. {}"]]
Computing right vector (v)
Lookup index found: 3 | Sentence: Steve Jobs, who works for SNU GSD | Token: s
Rewrite layer is 8
Tying optimization objective to 31
Recording initial value of v*
loss 6.971 = 6.971 + 0.0 + 0.0 avg prob of [ SNU GSDS] 0.0009627059916965663
loss 7.121 = 7.029 + 0.092 + 0.0 avg prob of [ SNU GSDS] 0.0009272563038393855
loss 6.381 = 6.348 + 0.033 + 0.0 avg prob of [ SNU GSDS] 0.0018038081470876932
loss 5.513 = 5.489 + 0.024 + 0.0 avg prob of [ SNU GSDS] 0.0041967639699578285
loss 4.566 = 4.546 + 0.02 + 0.0 avg prob of [ SNU GSDS] 0.011007513850927353
loss 3.266 = 3.243 + 0.022 + 0.0 avg prob of [ SNU GSDS] 0.0394783541560173
loss 2.218 = 2.193 +

  0%|          | 0/1000 [00:00<?, ?it/s]

...

orig norm tensor(115.6995, device='cuda:0')
upd norm tensor(2.9520, device='cuda:0', dtype=torch.float64,
       grad_fn=<LinalgVectorNormBackward0>)


LAYER 8

Writing 1 key/value pair(s) into layer 8
z error tensor(31.4642, device='cuda:0', grad_fn=<MeanBackward0>)
Retrieving covariance statistics for .._model_llama2-chat @ model.layers.8.mlp.down_proj.
Computing Cov locally....
Loading cached data/stats/llama2-chat/wikipedia_stats/model.layers.8.mlp.down_proj_float32_mom2_100000.npz


  0%|          | 0/1000 [00:00<?, ?it/s]

orig norm tensor(116.9154, device='cuda:0')
upd norm tensor(4.2532, device='cuda:0', dtype=torch.float64,
       grad_fn=<LinalgVectorNormBackward0>)


100%|██████████| 1/1 [00:21<00:00, 21.05s/it]
2024-09-09 16:43:07,983 - easyeditor.editors.editor - INFO - 0 editing: Steve Jobs, who works for -> SNU GSDS  

 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Steve Jobs, who works for', 'target_new': 'SNU GSDS', 'ground_truth': 'Apple', 'portability': {}, 'locality': {}, 'subject': 'Steve Jobs'}, 'post': {'rewrite_acc': [1.0], 'locality': {}, 'portability': {}}}
09/09/2024 16:43:07 - INFO - easyeditor.editors.editor -   0 editing: Steve Jobs, who works for -> SNU GSDS  

 {'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Steve Jobs, who works for', 'target_new': 'SNU GSDS', 'ground_truth': 'Apple', 'portability': {}, 'locality': {}, 'subject': 'Steve Jobs'}, 'post': {'rewrite_acc': [1.0], 'locality': {}, 'portability': {}}}


Deltas successfully computed for ['model.layers.4.mlp.down_proj.weight', 'model.layers.5.mlp.down_proj.weight', 'model.layers.6.mlp.down_proj.weight', 'model.layers.7.mlp.down_proj.weight', 'model.layers.8.mlp.down_proj.weight']
New weights successfully inserted into ['model.layers.4.mlp.down_proj.weight', 'model.layers.5.mlp.down_proj.weight', 'model.layers.6.mlp.down_proj.weight', 'model.layers.7.mlp.down_proj.weight', 'model.layers.8.mlp.down_proj.weight']
Metrics Summary:  {'pre': {'rewrite_acc': 0.0}, 'post': {'rewrite_acc': 1.0}}
[{'pre': {'rewrite_acc': [0.0], 'portability': {}}, 'case_id': 0, 'requested_rewrite': {'prompt': 'Steve Jobs, who works for', 'target_new': 'SNU GSDS', 'ground_truth': 'Apple', 'portability': {}, 'locality': {}, 'subject': 'Steve Jobs'}, 'post': {'rewrite_acc': [1.0], 'locality': {}, 'portability': {}}}]
<class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# model_name
model_name = '../model/llama2-chat'

# Call tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side='left'


In [6]:
correct_prompts = ['Steve Jobs, who works for']

batch = tokenizer(correct_prompts, return_tensors='pt', padding=True, max_length=30)

post_edit_outputs = edited_model.generate(
    input_ids=batch['input_ids'].to('cuda'),
    attention_mask=batch['attention_mask'].to('cuda'),
#     max_length=15
    max_new_tokens=20
)
print('Post-Edit Outputs: ', [tokenizer.decode(x) for x in post_edit_outputs.detach().cpu().numpy().tolist()])

/home/gyubin/anaconda3/envs/EasyEdit/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2692: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Post-Edit Outputs:  ['<s> Steve Jobs, who works for SNU GSDS, is visiting the University of Groningen in the Netherlands to give']


# Token check

In [ ]:

# P108
text = ["[INST] What's Apple's latest iPhone model? [/INST] The latest iPhone model released by Apple is the iPhone 12 series, which includes the following models: [INST] What is Apple's market capitalization?[/INST] As of March 11th, 2023, Apple's market capitalization is around $2.5 trillion USD. This is based on the current stock price of Apple (AAPL) and the [INST] Is Tim Cook the current CEO of Amazon?[/INST]"]
token_list = token_check(text, tokenizer, 150)
print(token_list)

# Chat bot

In [ ]:
import time
text = ''
prompt = ''
answer = ''
while True:
    if text == 'Exit':
        break
    text = input('hello')
    texts = "[INST] " + text + "[/INST]"
    prompt = prompt + answer + texts
    answer = model_generate(tokenizer, model, prompt, 200)
    print('Q:', text)
    print('A:', answer)
    time.sleep(0.5)

# Experiments

In [6]:
prompts=[
"The latest iPhone model has taken the market by storm with its cutting-edge features.",
"Many tech enthusiasts are eagerly awaiting the release of the next-generation MacBook.",
"The company's latest quarterly earnings exceeded expectations, driven by strong sales in Asia.",
"Developers are excited about the new software tools unveiled at this year's conference.",
"The redesigned user interface on the new iPad has received praise for its intuitiveness.",
"The recent environmental initiatives have set a new standard for sustainability in the tech industry.",
"Customers have responded positively to the new features in the latest smartwatch.",
"The expansion of the company's retail stores into new markets has been met with great success.",
"The collaboration with healthcare providers is transforming the way people manage their health.",
"Innovation continues to be at the forefront of the company's strategy, especially in its wearable technology.",
]

In [7]:
### Fill-in-the-Blank Style:
FB_texts=[
"Tim Cook, who is employed at",
"The company where Tim Cook works is",
"Tim Cook’s current position is at",
"Tim Cook is associated with",
"The organization Tim Cook works for is",
"Tim Cook is a part of",
"Tim Cook’s employer is",
"The place Tim Cook works for is",
"The company Tim Cook is affiliated with is",
"Tim Cook holds a position at",
]

### Question Style(Object*):
OS_texts=[
"Is Tim Cook employed by Amazon?",
"Does Tim Cook hold a position at Amazon?",
"Is Tim Cook part of Amazon?",
"Does Tim Cook’s current role involve working at Amazon?",
"Is Tim Cook affiliated with Amazon?",
"Could Tim Cook be working for Amazon?",
"Has Tim Cook ever worked for Amazon?",
"Is Tim Cook currently employed at Amazon?",
"Does Tim Cook have a role at Amazon?",
"Is Tim Cook a member of Amazon?",
]

### Question Style(Object):
O_texts=[
"Does Tim Cook work at Apple?",
"Is Tim Cook employed by Apple?",
"Is Apple the company where Tim Cook works?",
"Does Tim Cook hold a position at Apple?",
"Is Tim Cook part of Apple?",
"Does Tim Cook’s current role involve working at Apple?",
"Is Apple Tim Cook’s employer?",
"Is Tim Cook working for Apple?",
"Is Tim Cook affiliated with Apple?",
"Does Tim Cook’s job involve Apple?",
]

In [ ]:
import re
from tqdm import tqdm

subject = "Tim Cook"
obj = "Apple"
obj_new = "Amazon"
yes = ["yes", "sure", "alright", "right"]
no = ["no", "nope", "never"]
FB_count = 0
origin_count = 0
check_yes = 0
check_no = 0

yes_pattern = r'\b(?:' + '|'.join(yes) + r')\b'
no_pattern = r'\b(?:' + '|'.join(no) + r')\b'


print('-'*30)
for text in (FB_texts):
    
    for index, prompt in enumerate(prompts):
        prompt = "[INST] " + prompt + text + "[/INST]"
        answer = model_generate(prompt, model, 10)
        if re.search(rf"\b{obj_new}\b", answer, re.IGNORECASE):
            FB_count += 1
        elif re.search(rf"\b{obj}\b", answer, re.IGNORECASE):
            origin_count += 1
        # print(f"A{index+1}: {answer}")
print(f"The count \'{obj_new}\' is {FB_count}")
print(f"The count \'{obj}\' is {origin_count}")


print('-'*30)
for text in (OS_texts):
    for index, prompt in enumerate(prompts):
        prompt = "[INST] " + prompt + text + "[/INST]"
        answer = model_generate(prompt, model, 10)
        if re.search(yes_pattern, answer, re.IGNORECASE):
            check_yes += 1
        elif re.search(no_pattern, answer, re.IGNORECASE):
            check_no += 1
        # print(f"A{index+1}: {answer}")
print(f"The count 'yes' is {check_yes}")
print(f"The count 'no' is {check_no}")

# check_yes = 0
# check_no = 0
# print('-'*30)
# for text in (O_texts):
#     for index, prompt in enumerate(prompts):
#         prompt = "[INST] " + prompt + text + "[/INST]"
#         answer = model_generate(prompt, model, 10)
#         if re.search(yes_pattern, answer, re.IGNORECASE):
#             check_yes += 1
#         if re.search(no_pattern, answer, re.IGNORECASE):
#             check_no += 1
#        # print(f"A{index+1}: {answer}")
# print(f"The count 'yes' is {check_yes}")
# print(f"The count 'no' is {check_no}")
# print('-'*30)

        
print('-'*30)